Question 2: What time window will have the most players in it?

In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
library(purrr)
library(dplyr)
library(lubridate)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

sources:
https://r4ds.had.co.nz/dates-and-times.html
https://forum.posit.co/t/converting-column-types/157014

In [2]:
# Loading the data 
url_players <- "https://drive.google.com/uc?export=download&id=1Mw9vW0hjTJwRWx0bDXiSpYsO3gKogaPz"
url_sessions <- "https://drive.google.com/uc?export=download&id=14O91N5OlVkvdGxXNJUj5jIsV5RexhzbB"

#players <- read_csv(url_players)  --> dont seem to need this data set
sessions <- read_csv(url_sessions)

#head(players) 
head(sessions)

Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


hashedEmail,start_time,end_time,original_start_time,original_end_time
<chr>,<chr>,<chr>,<dbl>,<dbl>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,30/06/2024 18:12,30/06/2024 18:24,1.71977e+12,1.71977e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,17/06/2024 23:33,17/06/2024 23:46,1.71867e+12,1.71867e+12
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,25/07/2024 17:34,25/07/2024 17:57,1.72193e+12,1.72193e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,25/07/2024 03:22,25/07/2024 03:58,1.72188e+12,1.72188e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,25/05/2024 16:01,25/05/2024 16:12,1.71665e+12,1.71665e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,23/06/2024 15:08,23/06/2024 17:10,1.71916e+12,1.71916e+12


In [3]:
# wranging data to just the data interested in

select_sessions <- sessions|> 
                  select(start_time, end_time)
head(select_sessions)             

start_time,end_time
<chr>,<chr>
30/06/2024 18:12,30/06/2024 18:24
17/06/2024 23:33,17/06/2024 23:46
25/07/2024 17:34,25/07/2024 17:57
25/07/2024 03:22,25/07/2024 03:58
25/05/2024 16:01,25/05/2024 16:12
23/06/2024 15:08,23/06/2024 17:10


In [4]:
# convert the data from character to date-time data type
dttm_sessions <- select_sessions|> 
                  mutate(start_time = ymd_hms(start_time),
                         end_time = ymd_hms(end_time)) 
head(dttm_sessions)

start_time,end_time
<dttm>,<dttm>
2030-06-21 00:18:12,2030-06-21 00:18:24
2017-06-21 00:23:33,2017-06-21 00:23:46
2025-07-21 00:17:34,2025-07-21 00:17:57
2025-07-21 00:03:22,2025-07-21 00:03:58
2025-05-21 00:16:01,2025-05-21 00:16:12
2023-06-21 00:15:08,2023-06-21 00:17:10


In [5]:
# Mutating data frame to only have the start time, kinda unneeded

time_sessions <- dttm_sessions|> 
        mutate(start_time = format(start_time," %H:%M:%S"),
               end_time = format(end_time," %H:%M:%S"))
head(time_sessions)

start_time,end_time
<chr>,<chr>
00:18:12,00:18:24
00:23:33,00:23:46
00:17:34,00:17:57
00:03:22,00:03:58
00:16:01,00:16:12
00:15:08,00:17:10


In [18]:
#separate columns into day, hours, mins
separated_sessions <- dttm_sessions|> 
                      mutate(start_time_day = format(start_time,"%H"),
                             end_time_day = format(end_time,"%H"),
                             start_time_hours = format(start_time,"%M"),
                             end_time_hours = format(end_time,"%M"),
                             start_time_mins = format(start_time,"%S"),
                             end_time_mins = format(end_time,"%S")) |>
                     select(-start_time, -end_time)     
head(separated_sessions)

start_time_day,end_time_day,start_time_hours,end_time_hours,start_time_mins,end_time_mins
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
00,00,18,18,12,24
00,00,23,23,33,46
00,00,17,17,34,57
00,00,03,03,22,58
00,00,16,16,01,12
00,00,15,17,08,10


In [35]:
#make all of the values numerical
sessions_numeric <- separated_sessions|> 
                    map_df(as.numeric)
                     
head(sessions_numeric)

start_time_day,end_time_day,start_time_hours,end_time_hours,start_time_mins,end_time_mins
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,0,18,18,12,24
0,0,23,23,33,46
0,0,17,17,34,57
0,0,3,3,22,58
0,0,16,16,1,12
0,0,15,17,8,10


In [42]:

#find the "length" of the start and end times in minutes
#sessions_length <- sessions_numeric|> 
                  # mutate(start_time_length = start_time_hours + start_time_mins,
                          #end_time_length = end_time_hours + end_time_mins)

#head(sessions_length)

#check if the player "played" into the next day, this condition the players played on the same day 

same_day <- sessions_numeric|>
            filter(end_time_hours > start_time_hours)
head(same_day)


#check if the player "played" into the next day, this condition the players played to the next

next_day <- sessions_numeric|>
            filter(start_time_hours > end_time_hours)
head(next_day)


next_day_hours <- next_day|> 
                  mutate(start_time_played = 24 - start_time_hours)|>
                  mutate(hours_played = start_time_played + end_time_hours)
head(next_day_hours)
              

#find the total length of the hours of the session if played on the same day 
total_length_same <- same_day|> 
                         mutate(same_length = end_time_length - start_time_length)|>
                         select(same_length)
#head(total_length_same)


#check if the player "played" into the next day, this condition the players played to the next

next_day <- sessions_length|>
            filter(start_time_length > end_time_length)
#head(next_day)
              

#find the total length of the session if played on the same day 
total_length_next <- next_day|> 
                         mutate(next_length = end_time_length + start_time_length)|>
                         select(next_length)
#head(total_length_next)


start_time_day,end_time_day,start_time_hours,end_time_hours,start_time_mins,end_time_mins
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,0,15,17,8,10
0,0,2,3,32,12
0,0,0,3,51,15
0,0,0,1,21,35
0,0,2,3,58,21
0,0,2,4,25,5


start_time_day,end_time_day,start_time_hours,end_time_hours,start_time_mins,end_time_mins
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,0,23,0,23,14
0,0,21,0,26,0
0,0,23,0,30,23
0,0,22,0,38,31
0,0,23,1,22,50
0,0,21,2,53,5


start_time_day,end_time_day,start_time_hours,end_time_hours,start_time_mins,end_time_mins,start_time_played,hours_played
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,0,23,0,23,14,1,1
0,0,21,0,26,0,3,3
0,0,23,0,30,23,1,1
0,0,22,0,38,31,2,2
0,0,23,1,22,50,1,2
0,0,21,2,53,5,3,5


ERROR: [1m[33mError[39m in `mutate()`:[22m
[1m[22m[36mℹ[39m In argument: `same_length = end_time_length - start_time_length`.
[1mCaused by error:[22m
[33m![39m object 'end_time_length' not found


In [ ]:
hour_sessions_counted <- hour_sessions|> 
                         group_by(start_time)|>
                         summarise(count = n())
head(hour_sessions_counted)

In [ ]:
hour_plot <- hour_sessions_counted|> 
             ggplot(aes(x = start_time, y = count))+
             geom_point()
hour_plot

In [1]:
gender_female = case_when(gender == "Male" ~ 0,
gender == "Female" ~ 1,
...),
...)

ERROR: Error in parse(text = x, srcfile = src): <text>:3:5: unexpected ','
2: gender == "Female" ~ 1,
3: ...),
       ^
